# QWED

> [QWED](https://github.com/QWED-AI/qwed-verification) (**Q**uery **w**ith **E**vidence and **D**eterminism) is a neurosymbolic verification engine that combines Large Language Models with symbolic reasoning engines (SymPy, Z3, AST) to provide deterministic verification of mathematical calculations, logical statements, and code.

QWED bridges the gap between neural flexibility and symbolic precision, making AI outputs verifiable and trustworthy.

This notebook demonstrates how to use QWED as a tool in LangChain agents for automated verification.

## Installation

```bash
pip install qwed
```

For PII masking support (enterprise privacy):
```bash
pip install 'qwed[pii]'
```

## Setup

Set your API key for the LLM provider:

In [ ]:
import os

# Set your API key (OpenAI, Anthropic, or Google)
os.environ["OPENAI_API_KEY"] = "your-api-key-here"

## Basic Usage

### Simple Verification

In [ ]:
from langchain_community.tools import QWEDTool

# Initialize QWED tool
tool = QWEDTool(
    provider="openai",  # or "anthropic", "gemini"
    model="gpt-4o-mini"  # or "claude-3-haiku-20240307"
)

# Verify a mathematical calculation
result = tool.run("What is the derivative of x^2?")
print(result)

Expected output:
```
✅ VERIFIED: 2*x
Confidence: 100%
Method: symbolic
```

### More Examples

In [ ]:
# Verify an integral
result = tool.run("What is the integral of 2x?")
print(result)

# Verify logic
result = tool.run("Is (A AND NOT A) satisfiable?")
print(result)

# Check code security
code_snippet = '''
def process_user_input(data):
    return eval(data)  # Dangerous!
'''
result = tool.run(f"Is this code safe?\n{code_snippet}")
print(result)

## Using with LangChain Agents

QWED becomes powerful when integrated with agents - agents can automatically call QWED for verification:

In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain_openai import ChatOpenAI
from langchain_community.tools import QWEDTool

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Initialize tools (QWED for verification)
tools = [QWEDTool(provider="openai")]

# Create agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

### Example 1: Math Verification

In [ ]:
# Agent will automatically use QWED to verify
response = agent.run(
    "I need to verify: is the integral of 2x equal to x^2 + C?"
)
print(response)

### Example 2: Logic Verification

In [ ]:
response = agent.run(
    "Can you verify if the statement (A OR NOT A) is always true?"
)
print(response)

### Example 3: Code Security Check

In [ ]:
dangerous_code = '''
import os
def run_command(cmd):
    os.system(cmd)  # Potential security risk!
'''

response = agent.run(
    f"Please check if this code is safe to execute:\n{dangerous_code}"
)
print(response)

## Advanced Configuration

### Using Different LLM Providers

In [ ]:
# Using Anthropic Claude
tool_claude = QWEDTool(
    provider="anthropic",
    api_key=os.getenv("ANTHROPIC_API_KEY"),
    model="claude-3-haiku-20240307"
)

# Using Google Gemini
tool_gemini = QWEDTool(
    provider="gemini",
    api_key=os.getenv("GOOGLE_API_KEY"),
    model="gemini-pro"
)

### Enterprise: PII Masking for Privacy

In [ ]:
# Enable PII masking (requires qwed[pii])
tool_private = QWEDTool(
    provider="openai",
    mask_pii=True  # Automatically masks emails, SSNs, credit cards, etc.
)

# Sensitive data is automatically masked before sending to LLM
result = tool_private.run(
    "User email: john@example.com wants to calculate 2+2"
)
# LLM sees: "User email: <EMAIL_ADDRESS> wants to calculate 2+2"
print(result)

## What QWED Verifies

QWED provides deterministic verification for:

### 1. **Mathematics**
- Calculus (derivatives, integrals, limits)
- Algebra (equations, simplification)
- Linear algebra (matrices, vectors)
- Statistics (probability, distributions)

### 2. **Logic**
- Propositional logic (SAT solving)
- Boolean algebra
- Logical equivalence

### 3. **Code Security**
- Dangerous patterns (eval, exec, os.system)
- SQL injection risks
- Command injection
- Path traversal vulnerabilities

## How It Works

```
Your Query
    ↓
LLM Response (GPT-4, Claude, etc.)
    ↓
Symbolic Verification (SymPy, Z3, AST)
    ↓
✅ Deterministic Proof or ❌ Verification Failure
```

**Example:**
- Query: "What is the derivative of x^2?"
- LLM says: "2x"
- SymPy computes: `diff(x**2, x) = 2*x`
- QWED: ✅ MATCH! Verified with 100% confidence

## Learn More

- **GitHub**: [QWED-AI/qwed-verification](https://github.com/QWED-AI/qwed-verification)
- **PyPI**: [qwed](https://pypi.org/project/qwed/)
- **Documentation**: [docs.qwedai.com](https://docs.qwedai.com)
- **GitHub Action**: [QWED on Marketplace](https://github.com/marketplace/actions/qwed-neurosymbolic-verification)